In [1]:
import json
import time
import asyncio
import aiohttp
import requests
from ttnex_auth import TtnexAuth

In [2]:
api_key = "ljgOOkd22YcailhAWaeBOciVRNdTXhfYNR0"
url = "http://65.21.73.24:8000/api/v1"
ttnex_auth = TtnexAuth(api_key)
buy_order_id = ""
sell_order_id = ""
pair = "LTC-BTC"

In [3]:
async def balance():
    async with aiohttp.ClientSession() as client:
        params = {}
        data = {"params": params}

        data = ttnex_auth.generate_auth_dict(data)
        headers = ttnex_auth.get_headers()

        post_json = json.dumps(data)
        response = await client.post(f"{url}/balance", data=post_json, headers=headers)

        try:
            parsed_response = json.loads(await response.text())
        except Exception as e:
            raise IOError(f"Error parsing data from {url}. Error: {str(e)}")
        if response.status != 200:
            raise IOError(f"Error fetching data from {url}. HTTP status is {response.status}. "
                          f"Message: {parsed_response}")
        if not parsed_response["status"]:
            raise IOError(f"{url} API call failed, response: {parsed_response['errors']}")

        return parsed_response

In [4]:
async def create_order(side, price, amount, client_oid):
    async with aiohttp.ClientSession() as client:
        params = {"pair": pair,
                "side": side,
                "price": price,
                "amount": amount,
                "client_oid": client_oid}
        data = {"params": params}

        data = ttnex_auth.generate_auth_dict(data)
        headers = ttnex_auth.get_headers()

        post_json = json.dumps(data)
        response = await client.post(f"{url}/create-order", data=post_json, headers=headers)

        try:
            parsed_response = json.loads(await response.text())
        except Exception as e:
            raise IOError(f"Error parsing data from {url}. Error: {str(e)}")
        if response.status != 200:
            raise IOError(f"Error fetching data from {url}. HTTP status is {response.status}. "
                          f"Message: {parsed_response}")
        if not parsed_response["status"]:
            raise IOError(f"{url} API call failed, response: {parsed_response['errors']}")

        return parsed_response

In [5]:
async def open_orders():
    async with aiohttp.ClientSession() as client:
        params = {"pair": pair}
        data = {"params": params}

        params = ttnex_auth.generate_auth_dict(data)
        headers = ttnex_auth.get_headers()

        post_json = json.dumps(data)
        response = await client.post(f"{url}/open-orders", data=post_json, headers=headers)

        try:
            parsed_response = json.loads(await response.text())
        except Exception as e:
            raise IOError(f"Error parsing data from {url}. Error: {str(e)}")
        if response.status != 200:
            raise IOError(f"Error fetching data from {url}. HTTP status is {response.status}. "
                          f"Message: {parsed_response}")
        if not parsed_response["status"]:
            raise IOError(f"{url} API call failed, response: {parsed_response['errors']}")
        
        return parsed_response

In [6]:
async def cancel_order(order_id):
    async with aiohttp.ClientSession() as client:
        params = {"order_id": order_id,
                 "pair": pair}
        data = {"params": params}

        params = ttnex_auth.generate_auth_dict(data)
        headers = ttnex_auth.get_headers()

        post_json = json.dumps(data)
        response = await client.post(f"{url}/cancel-order", data=post_json, headers=headers)

        try:
            parsed_response = json.loads(await response.text())
        except Exception as e:
            raise IOError(f"Error parsing data from {url}. Error: {str(e)}")
        if response.status != 200:
            raise IOError(f"Error fetching data from {url}. HTTP status is {response.status}. "
                          f"Message: {parsed_response}")
        if not parsed_response["status"]:
            raise IOError(f"{url} API call failed, response: {parsed_response['errors']}")

        return parsed_response

In [7]:
async def order_status(order_id):
    async with aiohttp.ClientSession() as client:
        params = {"order_id": order_id}
        data = {"params": params}

        params = ttnex_auth.generate_auth_dict(data)
        headers = ttnex_auth.get_headers()

        post_json = json.dumps(data)
        response = await client.post(f"{url}/order-status", data=post_json, headers=headers)

        try:
            parsed_response = json.loads(await response.text())
        except Exception as e:
            raise IOError(f"Error parsing data from {url}. Error: {str(e)}")
        if response.status != 200:
            raise IOError(f"Error fetching data from {url}. HTTP status is {response.status}. "
                          f"Message: {parsed_response}")
        if not parsed_response["status"]:
            raise IOError(f"{url} API call failed, response: {parsed_response['errors']}")

        return parsed_response

In [8]:
async def book():
    async with aiohttp.ClientSession() as client:
        headers = ttnex_auth.get_headers()

        response = await client.get(f"{url}/book?pair={pair}", headers=headers)

        try:
            parsed_response = json.loads(await response.text())
        except Exception as e:
            raise IOError(f"Error parsing data from {url}. Error: {str(e)}")
        if response.status != 200:
            raise IOError(f"Error fetching data from {url}. HTTP status is {response.status}. "
                          f"Message: {parsed_response}")
        if not parsed_response["status"]:
            raise IOError(f"{url} API call failed, response: {parsed_response['errors']}")
        
        return parsed_response

In [9]:
async def error_handler():
    
    # clean the previous orders
    try:
        orders = await asyncio.wait_for(open_orders(), timeout=10.0)
        print(json.dumps(orders, indent=4))
        for order in orders["data"]["order_list"]:
            # cancel buy order
            try:
                result = await asyncio.wait_for(cancel_order(order["order_id"]), timeout=10.0)
            except asyncio.CancelledError:
                raise
            except asyncio.TimeoutError:
                print('timeout!')
    except asyncio.CancelledError:
        raise
    except asyncio.TimeoutError:
        print('timeout!')

In [10]:
if __name__ == "__main__":
    
    # clean the slate
    await error_handler()
    
    # check balance
    try:
        result = await asyncio.wait_for(balance(), timeout=10.0)
        print(json.dumps(result, indent=4))
    except asyncio.CancelledError:
        await asyncio.wait_for(error_handler(), timeout=10.0)
        raise
    except asyncio.TimeoutError:
        print('timeout!')
        
    # get order-book before placing order
    try:
        result = await asyncio.wait_for(book(), timeout=10.0)
        print(json.dumps(result, indent=4))
    except asyncio.CancelledError:
        await asyncio.wait_for(error_handler(), timeout=10.0)
        raise
    except asyncio.TimeoutError:
        print('timeout!')
        
    # create a buy order
    try:
        result = await asyncio.wait_for(create_order("buy", 0.0039, 1, "HBOT-B-order1"), timeout=10.0)
        print(json.dumps(result, indent=4))
        buy_order_id = result["data"]["order_id"]
    except asyncio.CancelledError:
        await asyncio.wait_for(error_handler(), timeout=10.0)
        raise
    except asyncio.TimeoutError:
        print('timeout!')
        
    # create a sell order
    try:
        result = await asyncio.wait_for(create_order("sell", 0.0041, 1.2, "HBOT-S-order1"), timeout=10.0)
        print(json.dumps(result, indent=4))
        sell_order_id = result["data"]["order_id"]
    except asyncio.CancelledError:
        await asyncio.wait_for(error_handler(), timeout=10.0)
        raise
    except asyncio.TimeoutError:
        print('timeout!')
        
    # get order-book after placing orders
    try:
        result = await asyncio.wait_for(book(), timeout=10.0)
        print(json.dumps(result, indent=4))
    except asyncio.CancelledError:
        await asyncio.wait_for(error_handler(), timeout=10.0)
        raise
    except asyncio.TimeoutError:
        print('timeout!')
        
    # get all open orders of user
    try:
        result = await asyncio.wait_for(open_orders(), timeout=10.0)
        print(json.dumps(result, indent=4))
    except asyncio.CancelledError:
        await asyncio.wait_for(error_handler(), timeout=10.0)
        raise
    except asyncio.TimeoutError:
        print('timeout!')
    
    # check buy order status
    try:
        result = await asyncio.wait_for(order_status(buy_order_id), timeout=10.0)
        print(json.dumps(result, indent=4))
    except asyncio.CancelledError:
        await asyncio.wait_for(error_handler(), timeout=10.0)
        raise
    except asyncio.TimeoutError:
        print('timeout!')
    
    # check sell order status
    try:
        result = await asyncio.wait_for(order_status(sell_order_id), timeout=10.0)
        print(json.dumps(result, indent=4))
    except asyncio.CancelledError:
        await asyncio.wait_for(error_handler(), timeout=10.0)
        raise
    except asyncio.TimeoutError:
        print('timeout!')
    
    # cancel buy order
    try:
        result = await asyncio.wait_for(cancel_order(buy_order_id), timeout=10.0)
        print(json.dumps(result, indent=4))
    except asyncio.CancelledError:
        await asyncio.wait_for(error_handler(), timeout=10.0)
        raise
    except asyncio.TimeoutError:
        print('timeout!')
    
    # cancel sell order
    try:
        result = await asyncio.wait_for(cancel_order(sell_order_id), timeout=10.0)
        print(json.dumps(result, indent=4))
    except asyncio.CancelledError:
        await asyncio.wait_for(error_handler(), timeout=10.0)
        raise
    except asyncio.TimeoutError:
        print('timeout!')
        
    # check buy order status
    try:
        result = await asyncio.wait_for(order_status(buy_order_id), timeout=10.0)
        print(json.dumps(result, indent=4))
    except asyncio.CancelledError:
        await asyncio.wait_for(error_handler(), timeout=10.0)
        raise
    except asyncio.TimeoutError:
        print('timeout!')
    
    # check sell order status
    try:
        result = await asyncio.wait_for(order_status(sell_order_id), timeout=10.0)
        print(json.dumps(result, indent=4))
    except asyncio.CancelledError:
        await asyncio.wait_for(error_handler(), timeout=10.0)
        raise
    except asyncio.TimeoutError:
        print('timeout!')
        
    print("All APIs OK")

{
    "status": true,
    "data": {
        "count": 0,
        "order_list": []
    }
}
{
    "status": true,
    "data": {
        "balances": [
            {
                "asset": "BTC",
                "balance": 0.00999709939,
                "on_order": 0,
                "pending_withdraw": 0,
                "available": 0.00999709939
            },
            {
                "asset": "ETH",
                "balance": 0.2,
                "on_order": 0,
                "pending_withdraw": 0,
                "available": 0.2
            },
            {
                "asset": "LTC",
                "balance": 1.5,
                "on_order": 0,
                "pending_withdraw": 0,
                "available": 1.5
            },
            {
                "asset": "USDT",
                "balance": 10000,
                "on_order": 0,
                "pending_withdraw": 0,
                "available": 10000
            },
            {
                "asset": "TTN"